In [1]:
import numpy as np # 数据处理最重要的模块
import pandas as pd # 数据处理最重要的模块
import scipy.stats as stats # 统计模块
import scipy
# import pymysql  # 导入数据库模块

from datetime import datetime # 时间模块
import statsmodels.formula.api as smf  # OLS regression

# import pyreadr # read RDS file

from matplotlib import style
import matplotlib.pyplot as plt  # 画图模块
import matplotlib.dates as mdates

from matplotlib.font_manager import FontProperties # 作图中文
from pylab import mpl
#mpl.rcParams['font.sans-serif'] = ['SimHei']
#plt.rcParams['font.family'] = 'Times New Roman'

#输出矢量图 渲染矢量图
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

from IPython.core.interactiveshell import InteractiveShell # jupyter运行输出的模块
#显示每一个运行结果
InteractiveShell.ast_node_interactivity = 'all'

#设置行不限制数量
#pd.set_option('display.max_rows',None)

#设置列不限制数量
pd.set_option('display.max_columns', None)

In [2]:
data = pd.read_csv('datasets/000001.csv')
data['Day'] = pd.to_datetime(data['Day'],format='%Y/%m/%d')
data.set_index('Day', inplace = True)
data.sort_values(by = ['Day'],axis=0, ascending=True)

,Preclose,Open,Highest,Lowest,Close,Volume,Money
Day,,,,,,,
1990-12-19,,96.050,99.980,95.790,99.980,126000.00,4.940000e+05
1990-12-20,99.98,104.300,104.390,99.980,104.390,19700.00,8.400000e+04
1990-12-21,104.39,109.070,109.130,103.730,109.130,2800.00,1.600000e+04
1990-12-24,109.13,113.570,114.550,109.130,114.550,3200.00,3.100000e+04
1990-12-25,114.55,120.090,120.250,114.550,120.250,1500.00,6.000000e+03
...,...,...,...,...,...,...,...
2024-09-24,2748.918,2770.754,2863.152,2761.372,2863.126,4776195.45,4.427953e+07
2024-09-25,2863.126,2901.419,2952.451,2889.048,2896.306,5682598.16,5.166981e+07
2024-09-26,2896.306,2893.745,3000.953,2889.014,3000.953,5763192.61,5.246691e+07


In [3]:
data_new = data['1995-01':'2024-09'].copy()
data_new['Close'] = pd.to_numeric(data_new['Close'])
data_new['Preclose'] = pd.to_numeric(data_new['Preclose'])
data_new['Raw_return'] = data_new['Close'] / data_new['Preclose'] - 1
data_new

,Preclose,Open,Highest,Lowest,Close,Volume,Money,Raw_return
Day,,,,,,,,
1995-01-03,647.870,637.720,647.710,630.530,639.880,23451800.00,1.806930e+08,-0.012333
1995-01-04,639.880,641.900,655.510,638.860,653.810,42222000.00,3.069230e+08,0.021770
1995-01-05,653.810,655.380,657.520,645.810,646.890,43012300.00,3.015330e+08,-0.010584
1995-01-06,646.890,642.750,643.890,636.330,640.760,48748200.00,3.537580e+08,-0.009476
1995-01-09,640.760,637.520,637.550,625.040,626.000,50985100.00,3.985190e+08,-0.023035
...,...,...,...,...,...,...,...,...
2024-09-24,2748.918,2770.754,2863.152,2761.372,2863.126,4776195.45,4.427953e+07,0.041547
2024-09-25,2863.126,2901.419,2952.451,2889.048,2896.306,5682598.16,5.166981e+07,0.011589
2024-09-26,2896.306,2893.745,3000.953,2889.014,3000.953,5763192.61,5.246691e+07,0.036131


In [4]:
Month_data = data_new.resample('ME')['Raw_return'].apply(lambda x: (1+x).prod() - 1).to_frame()
Month_data.reset_index(inplace = True)
Month_data.rename(columns = {'Day':'month'}, inplace = True)
Month_data.set_index('month', inplace = True)
Month_data

,Raw_return
month,
1995-01-31,-0.131631
1995-02-28,-0.023694
1995-03-31,0.177803
1995-04-30,-0.103552
1995-05-31,0.207922
...,...
2024-05-31,-0.005801
2024-06-30,-0.038684
2024-07-31,-0.009656


In [5]:
inflation = pd.read_csv('datasets/inflation.csv')
inflation['month'] = pd.to_datetime(inflation['month'],format='%Y/%m/%d')
inflation.set_index('month',inplace=True)
inflation.sort_values(by=['month'],axis=0,ascending=True)

,cpi
month,
1987-01-31,5.1
1987-02-28,5.4
1987-03-31,5.8
1987-04-30,6.7
1987-05-31,7.6
...,...
2024-05-31,0.3
2024-06-30,0.2
2024-07-31,0.5


In [6]:
market_variance = data_new.resample('ME').apply({
    'Raw_return':
    lambda x: sum(x**2)
})
market_variance.reset_index(inplace=True)
market_variance.rename(columns={'Day':'month','Raw_return':'RV'},inplace=True)
market_variance.set_index('month',inplace=True)
market_variance

# market_variance <- daily_data[,.(MV = sum(Raw_return^2)),by = 'month']

,RV
month,
1995-01-31,0.005695
1995-02-28,0.018086
1995-03-31,0.013378
1995-04-30,0.008281
1995-05-31,0.148387
...,...
2024-05-31,0.000915
2024-06-30,0.000642
2024-07-31,0.001441


In [8]:
cross = pd.read_csv('datasets/cross_section2023.csv')
from pandas.tseries.offsets import MonthEnd
cross['month'] = pd.to_datetime(cross['month'], format='%Y-%m-%d') + MonthEnd(1)
cross['to_v'] = pd.to_numeric(cross['to_v'])
cross['floatingvalue'] = pd.to_numeric(cross['floatingvalue'])
cross = cross.dropna(subset=['to_v','floatingvalue'])
cross = cross.dropna()
# 补齐股票代码 如果不满6位 在前面补上0
cross['Stkcd'] = cross['Stkcd'].apply(lambda x: '{:0>6}'.format(x)) # 6位股票代码
cross['w'] = 1
cross

,Stkcd,month,Rank,Freq,ret,floatingvalue,totalvalue,sizef,sizet,to_v,w
1,000001,1991-05-31,2,24.0,-0.128345,1.016010e+09,1.859497e+09,20.739149,21.343572,0.007087,1
2,000001,1991-06-30,3,23.0,-0.119551,9.007350e+08,1.648521e+09,20.618722,21.223144,0.001155,1
3,000001,1991-07-31,4,16.0,-0.137013,7.828100e+08,1.432695e+09,20.478401,21.082823,0.000230,1
4,000001,1991-08-31,5,15.0,-0.417680,6.748338e+08,1.346275e+09,20.329977,21.020607,0.072087,1
5,000001,1991-09-30,6,24.0,-0.039425,6.523394e+08,1.301399e+09,20.296075,20.986706,0.160940,1
...,...,...,...,...,...,...,...,...,...,...,...
752023,605599,2023-08-31,24,23.0,-0.027103,7.955351e+09,1.054667e+10,22.797111,23.079076,0.134479,1
752024,605599,2023-09-30,25,20.0,0.071030,8.530295e+09,1.130889e+10,22.866890,23.148855,0.137497,1
752025,605599,2023-10-31,26,17.0,-0.048696,8.125488e+09,1.077222e+10,22.818272,23.100237,0.086762,1
752026,605599,2023-11-30,27,22.0,-0.027956,7.908417e+09,1.048444e+10,22.791193,23.073159,0.062795,1


In [9]:
turnover = pd.DataFrame(cross.groupby(['month']).apply(
    lambda x:
    np.average(x['to_v'],weights=x['floatingvalue']) 
, include_groups=False))
turnover = turnover.rename(columns={0:'to'})
turnover

,to
month,
1991-01-31,0.052992
1991-02-28,0.038486
1991-03-31,0.014105
1991-04-30,0.026301
1991-05-31,0.034965
...,...
2023-08-31,0.259480
2023-09-30,0.201205
2023-10-31,0.203478


In [11]:
price = pd.read_csv('datasets/priceratio.csv')
price['month'] = pd.date_range(start='1995',end='2024',freq='ME')
price.set_index('month',inplace=True)
price

,pb,pe,pd
month,,,
1995-01-31,0.803009,2.763315,4.084782
1995-02-28,0.781950,2.741896,4.081370
1995-03-31,0.864841,2.844483,4.238190
1995-04-30,0.614429,2.603794,4.117053
1995-05-31,0.760117,2.746883,4.185583
...,...,...,...
2023-08-31,0.261365,2.563063,3.728079
2023-09-30,0.255715,2.557339,3.727724
2023-10-31,0.210664,2.544169,3.700984


In [12]:
reg_data = pd.merge(Month_data,market_variance,on = 'month')
reg_data = pd.merge(reg_data,inflation,on = 'month')
reg_data = pd.merge(reg_data,price,on='month')
reg_data = pd.merge(reg_data,turnover,on='month')
reg_data

,Raw_return,RV,cpi,pb,pe,pd,to
month,,,,,,,
1995-01-31,-0.131631,0.005695,24.1,0.803009,2.763315,4.084782,0.095181
1995-02-28,-0.023694,0.018086,22.4,0.781950,2.741896,4.081370,0.092256
1995-03-31,0.177803,0.013378,21.3,0.864841,2.844483,4.238190,0.394602
1995-04-30,-0.103552,0.008281,20.7,0.614429,2.603794,4.117053,0.314511
1995-05-31,0.207922,0.148387,20.3,0.760117,2.746883,4.185583,0.721733
...,...,...,...,...,...,...,...
2023-08-31,-0.052009,0.001546,0.1,0.261365,2.563063,3.728079,0.259480
2023-09-30,-0.003013,0.000884,0.0,0.255715,2.557339,3.727724,0.201205
2023-10-31,-0.029483,0.001068,-0.2,0.210664,2.544169,3.700984,0.203478


In [13]:
reg_data['lto'] = reg_data['to'].shift(1)
reg_data['lcpi'] = reg_data['cpi'].shift(2)
reg_data['lpd'] = reg_data['pd'].shift(1)
reg_data

,Raw_return,RV,cpi,pb,pe,pd,to,lto,lcpi,lpd
month,,,,,,,,,,
1995-01-31,-0.131631,0.005695,24.1,0.803009,2.763315,4.084782,0.095181,NaN,NaN,NaN
1995-02-28,-0.023694,0.018086,22.4,0.781950,2.741896,4.081370,0.092256,0.095181,NaN,4.084782
1995-03-31,0.177803,0.013378,21.3,0.864841,2.844483,4.238190,0.394602,0.092256,24.1,4.081370
1995-04-30,-0.103552,0.008281,20.7,0.614429,2.603794,4.117053,0.314511,0.394602,22.4,4.238190
1995-05-31,0.207922,0.148387,20.3,0.760117,2.746883,4.185583,0.721733,0.314511,21.3,4.117053
...,...,...,...,...,...,...,...,...,...,...
2023-08-31,-0.052009,0.001546,0.1,0.261365,2.563063,3.728079,0.259480,0.233906,0.0,3.767052
2023-09-30,-0.003013,0.000884,0.0,0.255715,2.557339,3.727724,0.201205,0.259480,-0.3,3.728079
2023-10-31,-0.029483,0.001068,-0.2,0.210664,2.544169,3.700984,0.203478,0.201205,0.1,3.727724


# 样本外预测 Out-Of-Sample

我们前面所做的所有回归叫做样本内回归（In Sample）。

但是遗憾的是，如果模型的设定是正确的话，真实的世界里，人们并不能使用上述回归结果估计出来的参数进行真实的预测。

他们只能站在自己所在的时间点上，用能够获得的数据，进行“符合当时”的回归，得到参数并进行预测。

所以，我们有必要在真实的预测时，使用每个时间节点能获取的数据进行预测，这就叫做样本外预测。


具体而言,如果我们的观测样本一共有T期，在时间$t$,我们做以下回归
$$
r_{k} = \alpha + \beta * x_{k-1} + \epsilon_{k}, k=0,1,2,3...,t
$$
得到在时间$t$的模型估计参数,$\alpha_{t}$和$\beta_{t}$。那么我们要在$t$时刻预测$t+1$时刻的收益率，应该使用如下公式
$$
r_{a,t+1} = \alpha_{t} + \beta_{t} * x_{t}
$$
能够计算得到我们在时间$t$预测出的下一期$t+1$的收益率。

一般来说，我们假如不使用任何模型和额外的信息，只有股票收益率自己的话，能够作出的最好的预测应该就是收益率自己的历史均值，所以我们可以使用一个基准模型就是预测出的收益率等于
$$
r_{b,t+1} = \frac{\sum_{i=1}^{t}{r_i}}{t}
$$

我们定义预测误差为

\begin{gathered}
\hat{u}_{a, t+1} = r_{a,k+1} - r_k\\
\hat{u}_{b, t+1} = r_{b,k+1} - r_k
\end{gathered}

那么比较两个模型的优劣可以计算如下的统计量：

$$
1 - \frac{\sum^{T}_{t}(\hat{u}_{a, k+1})^2}{\sum^{T}_{t}(\hat{u}_{b, k+1} )^2}
$$

该统计量如果为正的，意味着我们的模型预测的结果比历史均值模型预测的要更精确一点，也就是说变量$x$真的提供了关于预测$r$的信息。

还有两个常用的样本外预测的检验统计量为：

\begin{gathered}
E N C-N E W= (T-t) \frac{ \sum_{t}^{T}\left[\hat{u}_{b, k+1}^{2}-\hat{u}_{b, k+1} \hat{u}_{a, k+1}\right]}{\sum_{t}^{T} \hat{u}_{a, k+1}^{2}} \\
M S E-F=(T-t) \frac{\sum_{t}^{T}\left[\hat{u}_{b, k+1}^{2}-\hat{u}_{a, k+1}^{2}\right]}{ \sum_{t}^{T} \hat{u}_{a, k+1}^{2}}
\end{gathered}

其中$T-t$为样本外预测的次数。

In [14]:
reg_data[347:348]

,Raw_return,RV,cpi,pb,pe,pd,to,lto,lcpi,lpd
month,,,,,,,,,,
2023-12-31,-0.018067,0.001009,-0.3,0.194571,2.527086,3.699837,0.233746,0.273387,-0.2,3.702013


In [15]:
data = reg_data['2000-01':'2023-12'].copy()
model_pre = 0
mean_pre = 0

for i in range(int(len(data)/3), len(data) - 1):
    # 选择数据
    data_reg = data[0:i]
    model =smf.ols(formula='Raw_return ~ lcpi', data=data_reg).fit(displ=False)
    r_b = (model.predict(data[i:i+1]['lcpi']) - data[i:i+1]['Raw_return'])**2
    r_a = (np.mean(data_reg['Raw_return']) - data[i:i+1]['Raw_return'])**2
    r_b = r_b.values
    r_a = r_a.values
    model_pre = model_pre + r_b
    mean_pre = mean_pre + r_a

oos = 1 - model_pre/mean_pre
print("样本外R2是:",oos)

样本外R2是: [0.05261273]


In [20]:
data = reg_data['2000-01':'2023-12'].copy()

def out_of_sample(data,y,x,initial_sample_fractions):
    model_pre = 0
    mean_pre = 0

    initial_sample = int(len(data)*initial_sample_fractions)

    # 模型
    predictor_formula = "+".join(x)
    formula = f'{y} ~ {predictor_formula}'

    for i in range(initial_sample, len(data) - 1):
        # 选择数据
        data_reg = data[0:i]
        model =smf.ols(formula, data=data_reg).fit(displ=False)
        r_b = (model.predict(data[i:i+1][x]) - data[i:i+1]['Raw_return'])**2
        r_a = (np.mean(data_reg['Raw_return']) - data[i:i+1]['Raw_return'])**2
        r_b = r_b.values
        r_a = r_a.values
        model_pre = model_pre + r_b
        mean_pre = mean_pre + r_a

    oos = 1 - model_pre/mean_pre
    return oos
out_of_sample(data,'Raw_return',['lcpi'],0.5)
out_of_sample(data,'Raw_return',['lcpi','lpd'],0.5)
out_of_sample(data,'Raw_return',['lcpi','lpd','lto'],0.5)

array([-0.0156638])

array([-0.03216968])

array([-0.13802383])

In [25]:
sample_fractions = np.linspace(0.1,0.9,9)

results = {}

for i in sample_fractions:
    results[i] = out_of_sample(data,'Raw_return',['lcpi','lpd','lto'],i)

# 打印数值i
for i, j in results.items():
    print(round(i,1),j)

0.1 [0.00087384]
0.2 [0.03906411]
0.3 [0.02233515]
0.4 [-0.16457213]
0.5 [-0.13802383]
0.6 [-0.15985361]
0.7 [-0.15457813]
0.8 [-0.23164427]
0.9 [-0.31566645]


In [30]:
# Out of Sample Test
# benchmark model 是基准模型，为历史均值
# augment model 是比较模型，是我们要添加的x

# I want to write a function to calculate the out-of-sample R2


data = reg_data['2000-01':'2021-12'].copy()
benchmark = 0.00
augment = 0.00
ab = 0.00

for i in range(int(len(data) / 3), len(data) - 1):
    data_reg = data[0:i]
    model1 = smf.ols('Raw_return ~ lto', data_reg).fit(displ=False)
    r_a = (model1.predict(data[i:i + 1]['lto']) - data[i:i + 1]['Raw_return'])**2
    r_b = (np.mean(data[0:i]['Raw_return']) - data[i:i + 1]['Raw_return'])**2
    r_ab = (model1.predict(data[i:i + 1]['lto']) - data[i:i + 1]['Raw_return']) * (
        np.mean(data[0:i]['Raw_return']) - data[i:i + 1]['Raw_return'])
    r_a = r_a.values
    r_b = r_b.values
    r_ab = r_ab.values
    augment = augment + r_a
    benchmark += r_b
    ab += r_ab

oos = 1 - augment / benchmark
ENCNEW = ((benchmark - ab) / augment) * (len(data) - int(len(data) / 3))
MSEF = (benchmark - augment) / augment * (len(data) - int(len(data) / 3))

print('样本外的R方是', oos)
print('样本外的𝐸𝑁𝐶-𝑁𝐸𝑊是', ENCNEW)
print('样本外的MSE-F是', MSEF)

样本外的R方是 [-0.03454616]
样本外的𝐸𝑁𝐶-𝑁𝐸𝑊是 [2.76926819]
样本外的MSE-F是 [-5.87709315]


In [31]:
# benchmark model 是基准模型，为历史均值
# augment model 是比较模型，是我们要添加的x

data = reg_data['2000-01':'2021-12'].copy()
benchmark = 0.00
augment = 0.00
ab = 0.00

for i in range(int(len(data) / 3), len(data) - 1):
    data_reg = data[0:i]
    model1 = smf.ols('Raw_return ~ lto + lpd', data_reg).fit(displ=False)
    r_a = (model1.predict(data[i:i + 1][['lto','lpd']]) - data[i:i + 1]['Raw_return'])**2
    r_b = (np.mean(data[0:i]['Raw_return']) - data[i:i + 1]['Raw_return'])**2
    r_ab = (model1.predict(data[i:i + 1][['lto','lpd']]) - data[i:i + 1]['Raw_return']) * (
        np.mean(data[0:i]['Raw_return']) - data[i:i + 1]['Raw_return'])
    r_a = r_a.values
    r_b = r_b.values
    r_ab = r_ab.values
    augment = augment + r_a
    benchmark += r_b
    ab += r_ab

oos = 1 - augment / benchmark
ENCNEW = ((benchmark - ab) / augment) * (len(data) - int(len(data) / 3))
MSEF = (benchmark - augment) / augment * (len(data) - int(len(data) / 3))

print('样本外的R方是', oos)
print('样本外的𝐸𝑁𝐶-𝑁𝐸𝑊是', ENCNEW)
print('样本外的MSE-F是', MSEF)

样本外的R方是 [-0.0211109]
样本外的𝐸𝑁𝐶-𝑁𝐸𝑊是 [4.14118947]
样本外的MSE-F是 [-3.6387018]


In [32]:
data = reg_data['2000-01':'2021-12'].copy()
benchmark = 0.00
augment = 0.00
ab = 0.00

for i in range(int(len(data) / 3), len(data) - 1):
    data_reg = data[0:i]
    model1 = smf.ols('Raw_return ~ lto + lpd + lcpi', data_reg).fit(displ=False)
    r_a = (model1.predict(data[i:i + 1][['lto','lpd','lcpi']]) - data[i:i + 1]['Raw_return'])**2
    r_b = (np.mean(data[0:i]['Raw_return']) - data[i:i + 1]['Raw_return'])**2
    r_ab = (model1.predict(data[i:i + 1][['lto','lpd','lcpi']]) - data[i:i + 1]['Raw_return']) * (
        np.mean(data[0:i]['Raw_return']) - data[i:i + 1]['Raw_return'])
    r_a = r_a.values
    r_b = r_b.values
    r_ab = r_ab.values
    augment = augment + r_a
    benchmark += r_b
    ab += r_ab

oos = 1 - augment / benchmark
ENCNEW = ((benchmark - ab) / augment) * (len(data) - int(len(data) / 3))
MSEF = (benchmark - augment) / augment * (len(data) - int(len(data) / 3))

print('样本外的R方是', oos)
print('样本外的𝐸𝑁𝐶-𝑁𝐸𝑊是', ENCNEW)
print('样本外的MSE-F是', MSEF)

样本外的R方是 [0.02007166]
样本外的𝐸𝑁𝐶-𝑁𝐸𝑊是 [16.12179675]
样本外的MSE-F是 [3.60497068]


# 更具一般性的函数
修改输入数据和公式

In [33]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

def out_of_sample(data, response_var, predictor_vars):
    benchmark = 0.00
    augment = 0.00
    ab = 0.00

    # 构建模型公式
    predictor_formula = ' + '.join(predictor_vars)
    formula = f'{response_var} ~ {predictor_formula}'

    for i in range(int(len(data) / 3), len(data) - 1):
        data_reg = data.iloc[0:i]
        model = smf.ols(formula, data_reg).fit(displ=False)
        prediction = model.predict(data.iloc[i:i + 1][predictor_vars])
        actual = data.iloc[i:i + 1][response_var].values[0]

        r_a = (prediction - actual) ** 2
        r_b = (np.mean(data_reg[response_var]) - actual) ** 2
        r_ab = (prediction - actual) * (np.mean(data_reg[response_var]) - actual)

        augment += r_a.values
        benchmark += r_b
        ab += r_ab.values

    oos = 1 - augment / benchmark
    ENCNEW = ((benchmark - ab) / augment) * (len(data) - int(len(data) / 3))
    MSEF = (benchmark - augment) / augment * (len(data) - int(len(data) / 3))

    return oos, ENCNEW, MSEF


oos, ENCNEW, MSEF = out_of_sample(reg_data['2000-01':'2021-12'], 'Raw_return', ['lto','lpd'])
print('样本外的R方是', oos)
print('样本外的𝐸𝑁𝐶-𝑁𝐸𝑊是', ENCNEW)
print('样本外的MSE-F是', MSEF)

样本外的R方是 [-0.0211109]
样本外的𝐸𝑁𝐶-𝑁𝐸𝑊是 [4.14118947]
样本外的MSE-F是 [-3.6387018]


# 更具有一般性的函数

In [34]:
def out_of_sample(data, response_var, predictor_vars, initial_sample_fraction):
    benchmark = 0.00
    augment = 0.00
    ab = 0.00

    # 构建模型公式
    predictor_formula = ' + '.join(predictor_vars) # 将predictor_vars中的元素用+连接起来
    formula = f'{response_var} ~ {predictor_formula}' # 构建公式

    initial_sample_size = int(len(data) * initial_sample_fraction) # 计算初始样本的大小

    for i in range(initial_sample_size, len(data) - 1):
        data_reg = data.iloc[0:i] # 取出前i行的数据
        model = smf.ols(formula, data_reg).fit(displ=False) # 拟合模型
        prediction = model.predict(data.iloc[i:i + 1][predictor_vars]) # 预测值
        actual = data.iloc[i:i + 1][response_var].values[0] # 从data中取出第i行的response_var列的值

        r_a = (prediction - actual) ** 2 # 残差平方
        r_b = (np.mean(data_reg[response_var]) - actual) ** 2 # 均值平方
        r_ab = (prediction - actual) * (np.mean(data_reg[response_var]) - actual) # 残差乘均值

        augment += r_a.values   
        benchmark += r_b       
        ab += r_ab.values     

    oos = 1 - augment / benchmark
    ENCNEW = ((benchmark - ab) / augment) * (len(data) - initial_sample_size)
    MSEF = (benchmark - augment) / augment * (len(data) - initial_sample_size)

    return oos, ENCNEW, MSEF

# 使用函数
oos, ENCNEW, MSEF = out_of_sample(reg_data['2000-01':'2021-12'], 'Raw_return', ['lto','lpd'], initial_sample_fraction=3/4)
print('样本外的R方是', oos)
print('样本外的𝐸𝑁𝐶-𝑁𝐸𝑊是', ENCNEW)
print('样本外的MSE-F是', MSEF)


样本外的R方是 [-0.04354756]
样本外的𝐸𝑁𝐶-𝑁𝐸𝑊是 [-0.50062308]
样本外的MSE-F是 [-2.75420021]


# 字典
Python中的字典（Dictionary）是一种可变的容器模型，可以存储任意类型对象。字典的每个键值对（key-value pair）用冒号 : 分隔，每个键值对之间用逗号 , 分隔，整个字典包裹在花括号 {} 中。
键必须是唯一的，但值则不必。

In [35]:
# 创建字典:
my_dict = {'name': 'Hang', 'age': 35}
my_dict

{'name': 'Hang', 'age': 35}

In [36]:
another_dict = dict(name="Bob", age=30)
another_dict

{'name': 'Bob', 'age': 30}

In [37]:
# 访问元素:
# 使用键来访问对应的值。
print(my_dict['name']) 

Hang


In [38]:
# 修改元素:
# 可以直接通过键来修改对应的值。
my_dict['age'] = 26


In [39]:
# 添加新的键值对:
# 直接赋值一个新键及其值。
my_dict['address'] = 'Da Lian'

In [40]:
# 删除元素:
# 使用 del 语句或 pop 方法。
del my_dict['address']  # 删除键 'address'
age = my_dict.pop('age')  # 移除键 'age' 并返回其值


In [41]:
my_dict

{'name': 'Hang'}

In [42]:
# 检查键是否存在:
# 使用 in 关键字。
if 'name' in my_dict:
    print("Name is present in the dictionary")


Name is present in the dictionary


1. keys()：返回字典中所有键的列表。
2. values()：返回字典中所有值的列表。
3. items()：返回键值对的列表。
4. get()：获取键对应的值，如果键不存在，可以返回默认值。
5. update()：用另一个字典的键值对更新当前字典。

In [43]:
for key in my_dict:
    print(key, my_dict[key])



name Hang


In [44]:
for value in my_dict.values():
    print(value)

Hang


In [45]:
for key, value in my_dict.items():
    print(key, value)

name Hang


In [46]:
my_dict = {
    'fruits': ['apple', 'banana', 'cherry'],
    'numbers': [1, 2, 3],
    'colors': ['red', 'green', 'blue']
}

print(my_dict)

{'fruits': ['apple', 'banana', 'cherry'], 'numbers': [1, 2, 3], 'colors': ['red', 'green', 'blue']}


In [47]:
keys = my_dict.keys()  # 返回所有键
values = my_dict.values()  # 返回所有值
keys
values

dict_keys(['fruits', 'numbers', 'colors'])

dict_values([['apple', 'banana', 'cherry'], [1, 2, 3], ['red', 'green', 'blue']])

In [48]:
my_dict['numbers']

[1, 2, 3]

In [49]:
# 设置不同的初始样本比例
initial_sample_fractions = [0.3, 0.4, 0.5, 0.6, 0.7,0.8]

# 用于存储结果的字典
results = {}

for fraction in initial_sample_fractions:
    # 调用 out_of_sample 函数
    oos, ENCNEW, MSEF = out_of_sample(reg_data['2000-01':'2021-12'], 'Raw_return', ['lto','lpd'], initial_sample_fraction=fraction)
    
    # 将结果保存到字典
    results[fraction] = {
        'OOS': oos,
        'ENCNEW': ENCNEW,
        'MSEF': MSEF
    }

# 打印结果
for fraction, metrics in results.items():
    print(f"初始样本比例: {fraction}, 样本外的R方: {metrics['OOS']}, ENC-NEW: {metrics['ENCNEW']}, MSE-F: {metrics['MSEF']}")


初始样本比例: 0.3, 样本外的R方: [0.02035198], ENC-NEW: [8.95874762], MSE-F: [3.84333629]
初始样本比例: 0.4, 样本外的R方: [-0.04233821], ENC-NEW: [1.05501071], MSE-F: [-6.4583403]
初始样本比例: 0.5, 样本外的R方: [-0.09143245], ENC-NEW: [-2.02704548], MSE-F: [-11.05802095]
初始样本比例: 0.6, 样本外的R方: [-0.10868989], ENC-NEW: [-2.15524161], MSE-F: [-10.39165998]
初始样本比例: 0.7, 样本外的R方: [-0.2593602], ENC-NEW: [-5.90274597], MSE-F: [-16.47568047]
初始样本比例: 0.8, 样本外的R方: [-0.03087661], ENC-NEW: [-0.1194025], MSE-F: [-1.5874452]


In [50]:
results

{0.3: {'OOS': array([0.02035198]),
  'ENCNEW': array([8.95874762]),
  'MSEF': array([3.84333629])},
 0.4: {'OOS': array([-0.04233821]),
  'ENCNEW': array([1.05501071]),
  'MSEF': array([-6.4583403])},
 0.5: {'OOS': array([-0.09143245]),
  'ENCNEW': array([-2.02704548]),
  'MSEF': array([-11.05802095])},
 0.6: {'OOS': array([-0.10868989]),
  'ENCNEW': array([-2.15524161]),
  'MSEF': array([-10.39165998])},
 0.7: {'OOS': array([-0.2593602]),
  'ENCNEW': array([-5.90274597]),
  'MSEF': array([-16.47568047])},
 0.8: {'OOS': array([-0.03087661]),
  'ENCNEW': array([-0.1194025]),
  'MSEF': array([-1.5874452])}}

# 季度样本外

In [51]:
Qreg_data = reg_data.resample('Q').apply({
    'Raw_return':
    lambda x: np.exp(sum(np.log( 1 + x))) - 1,
    'to':
    lambda x: sum(x),
    'pd':
    lambda x: np.mean(x),
    'cpi':
    lambda x: sum(x),
})
Qreg_data['lto'] = Qreg_data['to'].shift(1)
Qreg_data['lpd'] = Qreg_data['pd'].shift(1)
Qreg_data['lcpi'] = Qreg_data['cpi'].shift(1)
Qreg_data

,Raw_return,to,pd,cpi,lto,lpd,lcpi
month,,,,,,,
2000-03-31,0.317318,1.860277,5.411622,0.3,NaN,NaN,NaN
2000-06-30,0.071041,1.310464,5.450551,0.3,1.860277,5.411622,0.3000
2000-09-30,-0.009310,1.021229,5.419918,0.8,1.310464,5.450551,0.3000
2000-12-31,0.085501,0.743491,5.593930,2.8,1.021229,5.419918,0.8000
2001-03-31,0.018954,0.544915,5.633003,2.0,0.743491,5.593930,2.8000
...,...,...,...,...,...,...,...
2020-12-31,0.079247,0.768780,4.086465,0.2,1.142121,4.058310,6.7715
2021-03-31,-0.008972,0.819679,4.128985,-0.1,0.768780,4.086465,0.2000
2021-06-30,0.043374,0.779270,4.143594,3.3,0.819679,4.128985,-0.1000


In [52]:
# benchmark model 是基准模型，为历史均值
# augment model 是比较模型，是我们要添加的x

data = Qreg_data['2000-01':'2021-12'].copy()
benchmark = 0.00
augment = 0.00
ab = 0.00

for i in range(int(len(data) / 3), len(data) - 1):
    data_reg = data[0:i]
    model1 = smf.ols('Raw_return ~ lto', data_reg).fit(displ=False)
    r_a = (model1.predict(data[i:i + 1]['lto']) - data[i:i + 1]['Raw_return'])**2
    r_b = (np.mean(data[0:i]['Raw_return']) - data[i:i + 1]['Raw_return'])**2
    r_ab = (model1.predict(data[i:i + 1]['lto']) - data[i:i + 1]['Raw_return']) * (
        np.mean(data[0:i]['Raw_return']) - data[i:i + 1]['Raw_return'])
    r_a = r_a.values
    r_b = r_b.values
    r_ab = r_ab.values
    augment = augment + r_a
    benchmark += r_b
    ab += r_ab

oos = 1 - augment / benchmark
ENCNEW = ((benchmark - ab) / augment) * (len(data) - int(len(data) / 3))
MSEF = (benchmark - augment) / augment * (len(data) - int(len(data) / 3))

print('样本外的R方是', oos)
print('样本外的𝐸𝑁𝐶-𝑁𝐸𝑊是', ENCNEW)
print('样本外的MSE-F是', MSEF)

样本外的R方是 [0.05493354]
样本外的𝐸𝑁𝐶-𝑁𝐸𝑊是 [6.44253614]
样本外的MSE-F是 [3.42947171]
